## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données

### **Lire le corpus** 

In [38]:
import shutil, re, pandas, random
from os import listdir, chdir, path
from pathlib import Path

acteur = 'chuqc'
langue = 'fr'
sous_corpus = False 
tag = ''

# Change the directory
if sous_corpus:
    base_path = '../03-corpus/2-sous-corpus/'
    file_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else: 
    base_path = '../03-corpus/2-data/1-fr/'
    file_path = path.join(base_path, acteur) + '.csv'

In [39]:
from pandas import *
with open(file_path, "r", encoding = "UTF-8") as f:
        data = read_csv(file_path)['text']
        #data = data[~data["url"].str.contains('pdf')] # Exclure les PDFs 

### **Nettoyage**

In [40]:
text = [str(t).strip('\n').lower().replace('’', '\'').replace("œ", "oe") for t in data]
punct = '[!#$%&\(\)*+,-/:;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'

text = [re.sub(punct, ' ', t).replace("' ", "'" ).replace("'", "'") for t in text]
text = [re.sub(spaces, ' ', t) for t in text]

In [41]:
nb_docs = len(text)

print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 1270 documents.


### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [42]:
n = round(1 * nb_docs)
corpus = random.sample(text, n)

print("On va travailler sur un échantillon correspondant à environ x % des documents du corpus, soit {} documents". format(len(corpus)))

corpus = " ".join(corpus)

On va travailler sur un échantillon correspondant à environ x % des documents du corpus, soit 1270 documents


**NLTK**\
https://www.nltk.org/ 

In [43]:
import nltk
#nltk.download(['popular'])

### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [44]:
file_path = '../04-filtrage/mwe_stopwords.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mwe_sw = [t.lower().strip('\n') for t in f.readlines()]

In [45]:
for mwe in mwe_sw:
    corpus = corpus.replace(mwe, ' MWE_STOP ').replace('  ', " ")

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [46]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

tokens = tokenizer_re.tokenize(corpus)

print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))

Avec le RegExpTokenizer, notre corpus contient 1311120 tokens.


In [47]:
temps = round(len(tokens) / 15000 / 60)
print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

Le POS tagging devrait prendre environ 1 minutes.


In [48]:
corpus = " ".join(tokens).replace("' ", "'")

In [49]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

### **Mapping POS Tags** (FRMG)

Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [50]:
#file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

#with open(file_path) as f:
#    csv = read_csv(f)

#treeTag = [term for term in csv['TreeTagger'].tolist()] 
#lefff = [term for term in csv['Lefff'].tolist()]

#mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [51]:
# tagged = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]
#  mapping[t.split('\t')[1]])
tagged = [(t.split('\t')[0], t.split('\t')[1]) for t in tagger.tag_text(corpus)]


#if len(t.split('\t')) >1

In [52]:
tagged[:25]

[('directions', 'NOM'),
 ('chu', 'VER:pper'),
 ('de', 'PRP'),
 ('québec', 'ADJ'),
 ('université', 'NOM'),
 ('laval', 'ADJ'),
 ('contenu', 'VER:pper'),
 ('de', 'PRP'),
 ('la', 'DET:ART'),
 ('page', 'NOM'),
 ('attention', 'NOM'),
 ('pour', 'PRP'),
 ('les', 'DET:ART'),
 ('personnes', 'NOM'),
 ('dont', 'PRO:REL'),
 ('le', 'DET:ART'),
 ('prochain', 'ADJ'),
 ('rendez', 'VER:pres'),
 ('vous', 'PRO:PER'),
 ('sera', 'VER:futu'),
 ('au', 'PRP:det'),
 ('centre', 'NOM'),
 ('intégré', 'VER:pper'),
 ('de', 'PRP'),
 ('cancérologie', 'NOM')]

### **Lemmatisation** (FrenchLefffLemmatizer)

https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

In [53]:
#from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
#lemmatizer = FrenchLefffLemmatizer()

#lemmas = []
#for term in tagged:
#    term_l = []
#    if lemmatizer.lemmatize(term[0], term[1]) == []:
#        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
    # elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
    #     term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

    # else:
    #     term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])
    
    # lemmas.append(term_l)

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [54]:
from nltk import everygrams
ngrammes = list(everygrams(tagged, min_len=2, max_len=6))
# ngrammes_lemmatized = list(everygrams(lemmas, min_len=2, max_len=5))

In [55]:
print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))

Avant filtrage, on a 6555520 ngrammes.


### **Extraction des patrons syntaxiques**

In [56]:
def extract_patterns(ngrammes):
    patterns = []

    for ng in ngrammes:
        phrase = [t[0] for t in ng]
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
        
    return patterns

In [57]:
phrases = extract_patterns(ngrammes)
# phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

In [58]:
# On calcule la distribution de fréquences de nos n-grammes
from nltk.probability import FreqDist

freq = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])
# freq_lemmatized = FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases_lemmatized])

In [59]:
freq.most_common(25)

[('de la', 11408),
 ("de l'", 8414),
 ('demande de', 7616),
 ('rendez vous', 6696),
 ('à la', 5794),
 ('de québec', 5226),
 ('tout sur', 4788),
 ("à l'", 4458),
 ('pour les', 4347),
 ('de consultation', 3816),
 ('demande de consultation', 3776),
 ('chu de', 3582),
 ('chu de québec', 3533),
 ('de rendez', 3453),
 ('de rendez vous', 3452),
 ("d'éthique", 3394),
 ('formulaire de', 3276),
 ("d'un", 3239),
 ('analyses en', 3228),
 ('des usagers', 3065),
 ('centre de', 3026),
 ('votre chirurgie', 2916),
 ('la recherche', 2914),
 ("d'évaluation", 2882),
 ('comité des', 2839)]

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [60]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

In [61]:
def filtrer_stopwords(x):
    return [term for term in x if not 'MWE_STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords]

In [62]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un chiffre

In [63]:
def filter_num(x):
    return [term for term in x if not term[0][0].isnumeric() and not term[0][-1].isnumeric()]

In [64]:
phrases = filter_num(phrases)

On retire les n-grammes qui débutent ou se terminent par token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [65]:
def filter_len(x):
    return [term for term in x if \
        len(term[0][0]) > 2 and len(term[0][0]) < 18 and \
        len(term[0][-1]) > 2 and len(term[0][-1]) < 18]

In [66]:
phrases = filter_len(phrases)

In [67]:
phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [68]:
for phrase in phrases:
    phrase.append(freq[phrase[0]])

In [69]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 1638119 occurrences de ngrammes.


In [70]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

,Expression,Patron syntaxique,Fréquence
0,directions chu,NOM VER:pper,1
1,directions chu de québec,NOM VER:pper PRP ADJ,1
2,directions chu de québec université,NOM VER:pper PRP ADJ NOM,1
3,chu de québec,VER:pper PRP ADJ,3533
4,chu de québec université,VER:pper PRP ADJ NOM,2367
...,...,...,...
1638114,enseignement et des affaires universitaires,NOM KON PRP:det NOM ADJ,270
1638115,affaires universitaires,NOM ADJ,270
1638116,chu de québec,VER:pper PRP NOM,3533
1638117,chu de québec untitled,VER:pper PRP NOM ADJ,44


In [71]:
def tabCSV(phrases, titre):
    base_path = '../04-filtrage/output/'
    tab = DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"]).drop_duplicates()
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    file_path = path.join(base_path, acteur, acteur)
    if sous_corpus:
       file_path = path.join(base_path, acteur, tag, tag)
    

    Path(file_path).mkdir(parents=True, exist_ok=True)

    tab.to_csv(file_path + titre)

    return tab.values.tolist()

In [72]:
phrases = tabCSV(phrases,'_n-grams.csv')
# phrases_lemmatized = tabCSV(phrases_lemmatized, '_n-grams-lemmatized.csv')

In [73]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(phrases)))

Après filtrage, on a 280253 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [74]:
file_path = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:200] #[:200] # # On peut aussi seulement prendree les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [75]:
terms = [t for t in phrases if t[1] in patterns] #and not 'NOM NOM' in t[1]] # 
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [76]:
terms[:50]

[['demande de consultation', 'NOM PRP NOM', 3776],
 ['chu de québec', 'VER:pper PRP NOM', 3533],
 ['comité des usagers', 'NOM PRP:det NOM', 2837],
 ['comité des usagers', 'NOM PRP:det ADJ', 2837],
 ["comité d'éthique", 'NOM PRP NOM', 2792],
 ['clinique interdisciplinaire de mémoire', 'NOM ADJ PRP NOM', 2695],
 ['mémoire cime', 'NOM NOM', 2692],
 ['québec université', 'NOM NOM', 2368],
 ['québec université', 'ADJ NOM', 2368],
 ['professionnels de la santé', 'NOM PRP DET:ART NOM', 2312],
 ['intervention en santé', 'NOM PRP NOM', 2236],
 ['évaluation des technologies', 'NOM PRP:det NOM', 2232],
 ["modes d'intervention", 'NOM PRP NOM', 2231],
 ["modes d'intervention en santé", 'NOM PRP NOM PRP NOM', 2226],
 ['enfant jésus', 'NOM NOM', 1978],
 ['hôpital saint', 'NOM ADJ', 1977],
 ["saint françois d'assise", 'ADJ NOM PRP NOM', 1977],
 ["hôpital de l'enfant", 'NOM PRP DET:ART NOM', 1966],
 ["hôpital de l'enfant jésus", 'NOM PRP DET:ART NOM NOM', 1965],
 ['implant cochléaire', 'NOM ADJ', 1849]

In [77]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 74 % des termes
On avait 280253 ngrammes, on en a maintenant 71662.


In [78]:
import pandas as pd

def extract_terms(liste_terms, titre):
    file_path = '../04-filtrage/output/'
    tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)

    if sous_corpus:
        file_path = path.join(file_path, acteur, tag, tag)

    else :
        file_path = path.join(file_path, acteur, acteur)

                    
    tab.to_csv(file_path + titre)

extract_terms(terms, '_terms.csv')
#extract_terms(terms_lemmatized, '_terms-lemmatized.csv')

In [79]:
terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])

In [80]:
terms_patterns = terms_patterns.to_dict('records')

In [81]:
dict_patterns = {}
for term in terms_patterns:
    exp = term['Expression']
    pattern = term['Structure syntaxique']
    dict_patterns[exp] = pattern

# dict_patterns

In [82]:
DataFrame(terms_patterns)

,Expression,Structure syntaxique,Fréquence
0,demande de consultation,NOM PRP NOM,3776
1,chu de québec,VER:pper PRP NOM,3533
2,comité des usagers,NOM PRP:det NOM,2837
3,comité des usagers,NOM PRP:det ADJ,2837
4,comité d'éthique,NOM PRP NOM,2792
...,...,...,...
71657,plaie rougeur,NOM NOM,1
71658,plaie rougeur enflure,NOM NOM NOM,1
71659,rougeur enflure,NOM NOM,1
71660,rougeur enflure de la plaie,NOM NOM PRP DET:ART NOM,1


### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [83]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [84]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 71662 ngrammes.


In [85]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2
from nltk import bigrams, trigrams, ngrams

In [86]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [87]:
def llr_ngrammes(n):
    llr = []

    for i in range(2, n+1):
        ngrammes = set([tuple(tokenizer_re.tokenize(term[0])) for term in terms if len(tokenizer_re.tokenize(term[0])) == i])
        fd = nltk.FreqDist(ngrams(tokens, n=i))
        fd_prior = nltk.FreqDist(ngrams(tokens, n=i-1))
        
        for t in ngrammes:
            c_prior = fd_prior[t[:i-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[i-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = fd[t]             # Le ngramme lui-même P(w1w2w3..wn)

            res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
            p = chi2.sf(res, 1) # 1 degrees of freedom
            #if res == float('-inf') :
            #    res = 50000

            if p < 0.005 or (res == float('-inf')):
                llr.append({'Collocation' : " ".join(t).replace("' ", "'"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p})

    return llr

In [88]:
terms = llr_ngrammes(6)

In [89]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len_prior - len(terms)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len_prior - len(terms)) / len_prior *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 6615 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 9 % de nos n-grammes.


In [90]:
DataFrame(terms).sort_values(by = "Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value
33271,demande de consultation,NOM PRP NOM,3776,-inf,1.000000
26245,chu de québec,VER:pper PRP NOM,3533,-inf,1.000000
30446,comité des usagers,NOM PRP:det ADJ,2837,-inf,1.000000
36854,comité d'éthique,NOM PRP NOM,2792,-inf,1.000000
50891,clinique interdisciplinaire de mémoire,NOM ADJ PRP NOM,2695,-inf,1.000000
...,...,...,...,...,...
26643,neurologie de hej,NOM PRP NOM,1,18.399623,0.000018
26645,adaptée au préalable,VER:pper PRP:det NOM,1,19.395162,0.000011
26646,transporté par évaq,VER:pper PRP NOM,1,21.033785,0.000005
26647,étude des voies,NOM PRP:det NOM,1,19.645812,0.000009


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; en tokenisant, on supprimer les frontières entre les mots qui sont des tirets ou autres caractères qu'un espace ou un apostrophe ; ça fait en sorte qu'on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [91]:
dfs = {term['Collocation']: len([doc for doc in text if term['Collocation'] in doc]) for term in terms}

In [92]:
max_df = round(0.85 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 85% des documents 
max_df

1080

In [93]:
test = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
test

789

In [94]:
zeros = {term for term in dfs if dfs[term] == 0}
dfs_100 = {term for term in dfs if dfs[term] >= max_df}

In [95]:
max_df = round(0.95 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans 95% des documents 
terms = [t for t in terms if dfs[t['Collocation']] < max_df ] #and dfs[t['Collocation']] > 0] 

In [96]:
colloc = [term['Collocation'] for term in terms]
len(colloc)

65047

In [97]:
for term in terms:
    term['DF'] = dfs[term['Collocation']] 
DataFrame(terms).sort_values(by="Fréquence", ascending=False)

,Collocation,Structure syntaxique,Fréquence,LLR,p-value,DF
33271,demande de consultation,NOM PRP NOM,3776,-inf,1.000000,531
26245,chu de québec,VER:pper PRP NOM,3533,-inf,1.000000,789
30446,comité des usagers,NOM PRP:det ADJ,2837,-inf,1.000000,544
36854,comité d'éthique,NOM PRP NOM,2792,-inf,1.000000,537
50891,clinique interdisciplinaire de mémoire,NOM ADJ PRP NOM,2695,-inf,1.000000,531
...,...,...,...,...,...,...
26643,neurologie de hej,NOM PRP NOM,1,18.399623,0.000018,1
26645,adaptée au préalable,VER:pper PRP:det NOM,1,19.395162,0.000011,1
26646,transporté par évaq,VER:pper PRP NOM,1,21.033785,0.000005,1
26647,étude des voies,NOM PRP:det NOM,1,19.645812,0.000009,1


In [ ]:
df = pd.DataFrame(terms)
df.sort_values(['Fréquence'], 
            axis=0,
            ascending=[False], 
            inplace=True)

output_path = path.join('../04-filtrage/output/', acteur, acteur + '_significant-collocations.csv') 
df.to_csv(output_path)

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'") for ng in ngrammes]

In [ ]:
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)


extrant.to_csv(file_path + '_KWIC' +'.csv')

### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
extr_mesh = tokenizer_mesh.tokenize([t['Collocation'] for t in terms])

# **MODIF**

In [ ]:
# MODIFICATION À APPORTER : dans l'extrant où on voit les termes mesh, ajouter la fréquence

termes_mesh[:15]

for t in termes_mesh:
    print(t, corpus.count(t))

NameError: name 'termes_mesh' is not defined

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [ ]:
file_path = '../04-filtrage/output/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in terms])

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
file_path = '../04-filtrage/output' 
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag) 

else :
    file_path = path.join(file_path, acteur, acteur)


df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')